In [1]:
import json
import pandas as pd
import os

In [5]:
SOURCE_DIR = '/home/sjb/Projects/Columbia/Applied_DL/CamelyonProject/data/'

In [6]:
label_file = 'zoom_1_256_256_partition_labels.json'
grayscale_file = 'zoom_1_256_256_partition_grayscale_info.json'
train_test_split_file = 'train_val_test_split.json'

In [7]:
label_df = pd.read_json(os.path.join(SOURCE_DIR, 
                                     'zoom_1_256_256_partition',
                                     'meta',
                                     label_file), dtype=False)
grayscale_df = pd.read_json(os.path.join(SOURCE_DIR, 
                                         'zoom_1_256_256_partition',
                                         'meta',
                                         grayscale_file), dtype=False)

In [8]:
train_test_split_df = pd.read_json(os.path.join(SOURCE_DIR,
                                                train_test_split_file), dtype=False)

In [9]:
label_df.shape, grayscale_df.shape, train_test_split_df.shape

((1266294, 3), (1266294, 3), (21, 2))

In [10]:
all_df = grayscale_df.merge(
    label_df.drop('img_id', axis=1), on='file_name').merge(
        train_test_split_df, on='img_id')

In [11]:
all_df.shape

(1266294, 5)

In [13]:
grayscale_df.shape

(1266294, 3)

In [14]:
GRAYSCALE_TOL = 0.4

In [15]:
all_df['include'] = (all_df['non_gray_ratio'] >= GRAYSCALE_TOL).astype(int)

In [16]:
all_df['include'].value_counts()

0    1163719
1     102575
Name: include, dtype: int64

In [17]:
all_df.head()

,file_name,img_id,non_gray_ratio,label,type,include
0,tumor_slide_001_split_249_24.png,001,0.016266,0,val,0
1,tumor_slide_001_split_31_127.png,001,0.004410,0,val,0
2,tumor_slide_001_split_108_188.png,001,0.000320,0,val,0
3,tumor_slide_001_split_406_51.png,001,0.000000,0,val,0
4,tumor_slide_001_split_160_72.png,001,0.720123,0,val,1


In [18]:
mask = (all_df['label'] == 1) & (all_df['include'] == 0) 

In [22]:
list_of_records = all_df.to_dict('records')

In [23]:
output_filename = 'all_info_0.4_grayscale_tol.json'

In [24]:
with open(os.path.join(SOURCE_DIR, 
                       'zoom_1_256_256_partition',
                       'meta',
                       output_filename), 'w') as f:
    json.dump(list_of_records, f, indent=4)